### Donut conversion to ONNX
You need to install transformers to this particular commit,

In [ ]:
!python -m pip install --upgrade git+https://github.com/huggingface/transformers.git@2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3

In [ ]:
# !pip uninstall onnxruntime-training
# !pip install onnxruntime-gpu
# !pip install transformers[onnx] tokenizers sentencepiece
# !pip install optimum[exporters]

Check if the code has been added to an stable version, it seems that commit has been merged already,

https://github.com/huggingface/transformers/pull/19254

I tried different tolerances, I remember we didn't need to go as large as 1e-2, but something in between 1e-2 and 1e-3, I tried all these(don't remember exactly which one was the successful one)

In [ ]:
## DONUT: DOCVQA
model_ckpt = "naver-clova-ix/donut-base-finetuned-docvqa"
!python -m transformers.onnx --model={model_ckpt} --feature=vision2seq-lm onnx/docvqa --atol 7e-3
# if there is an error try modifying atol in [1e-3, 7e-3]

## DONUT: CLASSIFIER RVLCDIP
model_ckpt = "naver-clova-ix/donut-base-finetuned-rvlcdip"
!python -m transformers.onnx --model={model_ckpt} --feature=vision2seq-lm onnx/rvlcdip --atol 7e-3
# if there is an error try modifying atol in [1e-3, 7e-3]

Framework not requested. Using torch to export to ONNX.
C:\Users\MSI\anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Unsupported PyTorch version for this model. Minimum required is 1.11, got: 1.10.1+cu113
Using framework PyTorch: 1.10.1+cu113
C:\Users\MSI\anaconda3\lib\site-packages\torch\onnx\utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
C:\Users\MSI\anaconda3\lib\site-packages\torch\onnx\utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTor

### Quantizing
#### decoder

In [ ]:
!olive optimize --model_path decoder_model.onnx --input_names input_ids,attention_mask,encoder_hidden_states --input_shapes [[-1,-1],[-1,-1],[-1,-1,1024]] --quantization_enabled --dynamic_batching_size 2

#### encoder/decoder with optimum

In [ ]:
from optimum.onnxruntime import AutoQuantizationConfig
from onnxruntime.quantization.qdq_quantizer import QDQQuantizer
from onnxruntime.quantization.onnx_quantizer import ONNXQuantizer
from optimum.onnxruntime import ORTQuantizableOperator
import onnx

quantization_config = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
quantizer_factory = QDQQuantizer if False else ONNXQuantizer

# here pick either model
path = "./decoder.onnx"
onnx_model = onnx.load(path)
quantizer = quantizer_factory(
                model=onnx_model,
                static=quantization_config.is_static,
                per_channel=quantization_config.per_channel,
                mode=quantization_config.mode,
                weight_qType=quantization_config.weights_dtype,
                activation_qType=quantization_config.activations_dtype,
                tensors_range=None,
                reduce_range=quantization_config.reduce_range,
                nodes_to_quantize=quantization_config.nodes_to_quantize,
                nodes_to_exclude=quantization_config.nodes_to_exclude,
                op_types_to_quantize=[
                    operator.value if isinstance(operator, ORTQuantizableOperator) else operator
                    for operator in quantization_config.operators_to_quantize
                ],
                extra_options={
                    "WeightSymmetric": quantization_config.weights_symmetric,
                    "ActivationSymmetric": quantization_config.activations_symmetric,
                    "EnableSubgraph": False,
                    "ForceSymmetric": quantization_config.activations_symmetric
                    and quantization_config.weights_symmetric,
                    "AddQDQPairToWeight": quantization_config.qdq_add_pair_to_weight,
                    "DedicatedQDQPair": quantization_config.qdq_dedicated_pair,
                    "QDQOpTypePerChannelSupportToAxis": quantization_config.qdq_op_type_per_channel_support_to_axis,
                },
            )

quantizer.quantize_model()
quantizer.model.save_model_to_file("quantized_by_optimum.onnx")

### Handling the memory issues
You should go to this file,

/home/ubuntu/.local/lib/python3.7/site-packages/transformers/onnx/convert.py

in method `validate_model_outputs()`

and patch things similar to the changes here.
This is just reducing the batch size from 3 to 1. I we really want to try 3 different examples, we could just run the validation 3 times, each time with batch size 1, so we don't kill the memory.

In [ ]:
    # generate inputs with a different batch_size and seq_len that was used for conversion to properly test
    # dynamic input shapes.
    if is_torch_available() and issubclass(type(reference_model), PreTrainedModel):
        reference_model_inputs = config.generate_dummy_inputs(
            preprocessor,
            batch_size=1, #config.default_fixed_batch + 1,
            seq_length=config.default_fixed_sequence + 1,
            framework=TensorType.PYTORCH,
        )
    else:
        reference_model_inputs = config.generate_dummy_inputs(
            preprocessor,
            batch_size=1, #config.default_fixed_batch + 1,
            seq_length=config.default_fixed_sequence + 1,
            framework=TensorType.TENSORFLOW,
        )
